In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt
import ta
import yfinance as yf
import seaborn as sns
import numpy as np
from queue import Queue
import holidays
import scipy as sp
from itertools import combinations
from pymongo import MongoClient
import swifter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

C:\Users\thede\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from helper.client import *
from helper.machinelearner import *
from helper.dbmgr import *
from helper.dataprocessor import *

# Create an instance of CoinbaseClient
client = CoinbaseClient(api_key, api_secret)

In [3]:
db_uri = 'mongodb://localhost:27017/'  
db_name = 'crypto_data'  

# Get and format data from CB

In [4]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

print(x.inserted_id)

65d746ebdaffcc9eeec13138


In [5]:
unique_pids = client.get_unique_product_ids()
unique_pids = [f"{i}-{'USDC'}" for i in unique_pids]

start_time = datetime(2022, 10, 1)
# start_time = datetime(2024, 2, 1)
end_time = datetime(2024, 2, 22)

db_manager = DatabaseManager(start_time=start_time, end_time=end_time)
print(db_manager.check_row_counts())

db_manager.main(unique_pids)

Checking the number of documents in each collection...
Collection 'crypto_candles_fifteen_minute' has 404485 documents.
Collection 'crypto_candles_thirty_minute' has 219059 documents.
Collection 'crypto_candles_one_hour' has 114516 documents.
Collection 'crypto_candles_two_hour' has 58633 documents.
Collection 'crypto_candles_six_hour' has 19660 documents.
Collection 'crypto_candles_one_day' has 4926 documents.
None
Preparing to fetch and store data...


# Technical Indicator creation

In [ ]:
fe = FeatureEngineer(db_name)
processed_data = fe.run()

Processing Collections:   0%|          | 0/6 [00:00<?, ?it/s]

Processing Collections: 100%|██████████| 6/6 [00:36<00:00,  6.08s/it]


# Preprocessing creation
- Labels
- Scaling
- Normalisation
- Stationary features

In [ ]:
data_preparator = DataPreparator(db_name)

# Prepare the data
final_data, non_stationary_columns = data_preparator.prepare_data()

BulkWriteError: batch op errors occurred, full error: {'writeErrors': [{'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: crypto_data.ML_features index: _id_ dup key: { _id: ObjectId('65d5a03f9862b969366a8c7f') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('65d5a03f9862b969366a8c7f')}, 'op': {'_id': ObjectId('65d5a03f9862b969366a8c7f'), 'instrument': 'MEDIA-USDC', 'start': Timestamp('2024-02-14 16:00:00'), 'close': 17.43, 'end': Timestamp('2024-02-14 16:15:00'), 'granularity': 'FIFTEEN_MINUTE', 'high': 17.69, 'low': 17.33, 'open': '17.67', 'volume': 423.481, 'granularity_fifteen': 'fifteen', '9d_ma_fifteen': 17.61888888888889, '20d_ma_low_fifteen': 17.4255, '20d_ma_high_fifteen': 17.6215, 'cci_fifteen': -120.6572769953042, 'close_fifteen': 17.43, 'end_fifteen': Timestamp('2024-02-14 16:15:00'), 'high_fifteen': 17.69, 'low_fifteen': 17.33, 'macd_fifteen': -0.06147583861654837, 'macd_signal_fifteen': -0.06338968199772374, 'obv_fifteen': 423.481, 'o

# Machine Learning on one set of parameters
- Grid Search for best parameters
- view feature importance on the best set and trim

# Save best model for backtesting

# Backtester